#### Zoo Model

Descrição: O dataset contém diversas informações sobre animais que devem ser classificados em 7 classes ("Mammal",
    Bird,
    Reptile,
    Fish,
    Amphibian,
    Insect,
    Invertebrate) para 101 instâncias.

### Imports

In [16]:

import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import wisardpkg as wp


### Carregando o Dataset Car Evaluation

In [17]:
zoo_model = fetch_ucirepo(id=111)
X_original = zoo_model.data.features
y_original = zoo_model.data.targets.iloc[:, 0]


print(f"Dimensão inicial de X: {X_original.shape}")
print(f"Colunas de X e seus tipos de dados:\n{X_original.dtypes}")
print(f"\nPrimeiras 5 linhas de X:\n{X_original.head()}")
print(f"\nClasses de y:\n{y_original.value_counts()}")


Dimensão inicial de X: (101, 16)
Colunas de X e seus tipos de dados:
hair        int64
feathers    int64
eggs        int64
milk        int64
airborne    int64
aquatic     int64
predator    int64
toothed     int64
backbone    int64
breathes    int64
venomous    int64
fins        int64
legs        int64
tail        int64
domestic    int64
catsize     int64
dtype: object

Primeiras 5 linhas de X:
   hair  feathers  eggs  milk  airborne  aquatic  predator  toothed  backbone  \
0     1         0     0     1         0        0         1        1         1   
1     1         0     0     1         0        0         0        1         1   
2     0         0     1     0         0        1         1        1         1   
3     1         0     0     1         0        0         1        1         1   
4     1         0     0     1         0        0         1        1         1   

   breathes  venomous  fins  legs  tail  domestic  catsize  
0         1         0     0     4     0         0      

### Pré-processamento

In [18]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # sparse_output=False retorna array denso
# sparse_output=False é o mesmo que chamar .toarray() depois.
X_encoded_dense = encoder.fit_transform(X_original)

print(f"Dimensão de X após One-Hot Encoding: {X_encoded_dense.shape}")
print(f"Tipo dos elementos após One-Hot Encoding: {X_encoded_dense.dtype}")
print(f"Primeiras 2 linhas de X_encoded_dense:\n{X_encoded_dense[:2]}")

Dimensão de X após One-Hot Encoding: (101, 36)
Tipo dos elementos após One-Hot Encoding: float64
Primeiras 2 linhas de X_encoded_dense:
[[0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0.
  0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1.]
 [0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0.
  0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1.]]


### Dividindo os Dados em Treino e Teste

In [19]:
X_encoded_uint8 = X_encoded_dense.astype(np.uint8)
y_str = y_original.astype(str).tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded_uint8, y_str, test_size=0.7, random_state=42, stratify=y_str
)

print(f"Tamanho do conjunto de treino (X_train): {X_train.shape[0]} amostras, {X_train.shape[1]} bits")
print(f"Tamanho do conjunto de teste (X_test): {X_test.shape[0]} amostras, {X_test.shape[1]} bits")
print(f"Tipo dos elementos de X_train: {X_train.dtype}")

Tamanho do conjunto de treino (X_train): 30 amostras, 36 bits
Tamanho do conjunto de teste (X_test): 71 amostras, 36 bits
Tipo dos elementos de X_train: uint8


### Treinando o Modelo WiSARD

In [20]:
addressSize = 3 

wsd = wp.Wisard(addressSize, ignoreZero=False, verbose=True)

print(f"WiSARD inicializado com addressSize={addressSize}")
wsd.train(X_train, y_train)

predictions_str = wsd.classify(X_test)

y_pred_np = np.array(predictions_str)
y_test_np = np.array(y_test) 

WiSARD inicializado com addressSize=3
training 30 of 30
classifying 71 of 71of 71


### Relatório de Classificação

In [21]:
print(classification_report(
    y_test_np, y_pred_np,
    labels=np.unique(y_test),
))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        29
           2       1.00      1.00      1.00        14
           3       1.00      0.33      0.50         3
           4       0.75      1.00      0.86         9
           5       1.00      0.67      0.80         3
           6       1.00      0.67      0.80         6
           7       0.78      1.00      0.88         7

    accuracy                           0.93        71
   macro avg       0.93      0.81      0.83        71
weighted avg       0.95      0.93      0.92        71



### WiSARD x MLP

O modelo da WiSARD alcançou resultados superiores aos da MLP em todas as métricas de avaliação.